# Import this (important)

In [ ]:
# Install the latest version of author's repo neural ode implementation
!git clone https://github.com/rtqichen/torchdiffeq.git
!cd torchdiffeq && pip install -e .
!ls torchdiffeq/torchdiffeq

Cloning into 'torchdiffeq'...
remote: Enumerating objects: 1138, done.
remote: Counting objects: 100% (434/434), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 1138 (delta 256), reused 400 (delta 240), pack-reused 704
Receiving objects: 100% (1138/1138), 8.29 MiB | 8.71 MiB/s, done.
Resolving deltas: 100% (682/682), done.
Obtaining file:///content/torchdiffeq
  Running setup.py develop for torchdiffeq
_impl  __init__.py


# New Section

# Rough

## Libraries

In [ ]:
import os
import sys
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot
import matplotlib.pyplot as plt

import time
import datetime
import argparse
import numpy as np
import pandas as pd
from random import SystemRandom
from sklearn import model_selection

import torch
import torch.nn as nn
from torch.nn.functional import relu
import torch.optim as optim


In [ ]:
import lib.utils as utils
from lib.plotting import *

from lib.rnn_baselines import *
from lib.ode_rnn import *
from lib.create_latent_ode_model import create_LatentODE_model
from lib.parse_datasets import parse_datasets
from lib.ode_func import ODEFunc, ODEFunc_w_Poisson
from lib.diffeq_solver import DiffeqSolver
from mujoco_physics import HopperPhysics

from lib.utils import compute_loss_all_batches


import os
import numpy as np

import torch
import torch.nn as nn

import lib.utils as utils
from lib.diffeq_solver import DiffeqSolver
from generate_timeseries import Periodic_1d
from torch.distributions import uniform

from torch.utils.data import DataLoader
from mujoco_physics import HopperPhysics
from physionet import PhysioNet, variable_time_collate_fn, get_data_min_max
from person_activity import PersonActivity, variable_time_collate_fn_activity

from sklearn import model_selection
import random

## Parameters

In [ ]:
n = 1000  # size of dataset
niters = 500  # number of iterations
lr = 1e-3  # Starting Learning Rate
b = 50  # batch_size
viz = True  # show plots while training
# save: path for saving checkpoints
# load: ID of the experiment to load for evaluation
random_seed = 1991 # random seed
# dataset = Dataset to load. Available: physionet, activity, hopper, periodic
sample_tp = None # number of time points to sub-sample.  If > 1, subsample exact number of points. If the number is in [0,1], take a percentage of available points per time series. If None, do not subsample
c = None # Cut out the section of the timeline of the specified length (in number of points
# quantization: Quantization on the physionet dataset." "Value 1 means quantization by 1 hour, value 0.1 means quantization by 0.1 hour = 6 min
latent_ode = None # latent ode with ode-rnn encoder
# z0-encoder = odernn # Type of encoder for Latent ODE model: odernn or rnn
# classic-rnn
# rnn-cell
# input-decay
# ode-rnn # Run ODE-RNN baseline: RNN-style that sees true points at every point. Used for interpolation only.
# rnn-vae  # Run RNN baseline: seq2seq model with sampling of the h0 and ELBO loss.
latents = 10 # size of the latent state
rec_dims = 20 # dimensionality of the recognition model (ODE or RNN)
rec_layers = 1 # Number of layers in ODE func in recognition ODE
gen_layers = 1 # Number of layers in ODE func in generative ODE
units = 100 # Number of units per layer in ODE func
gru_units = 100 # Number of units per layer in each of GRU update networks
poisson = 'store_true' # Model poisson-process likelihood for the density of events in addition to reconstruction
classif = 'store_true' # Include binary classification loss -- used for Physionet dataset for hospiral mortality
linear_classif = 'store_true' # If using a classifier, use a linear classifier instead of 1-layer NN
extrap = 'store_true' # Set extrapolation mode. If this flag is not set, run interpolation mode
timepoints = 100 # total number of timepoints
max_tp = 5. # We subsample points in the interval [0, args.max_tp]
noise_weight = 0.01 # Noise amplitude for generated traejctories
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
torch.manual_seed(random_seed)
np.random.seed(random_seed)

## Data

In [ ]:
from torch.distributions import uniform
n_total_tp = 100
max_t_extrap = max_tp/timepoints * 100

In [ ]:
distribution = uniform.Uniform(torch.Tensor([0.0]),torch.Tensor([max_t_extrap]))
time_steps_extrap =  distribution.sample(torch.Size([n_total_tp-1]))[:,0]
print(time_steps_extrap)
print(time_steps_extrap.shape)
time_steps_extrap = torch.cat((torch.Tensor([0.0]), time_steps_extrap))
print(time_steps_extrap.shape)
time_steps_extrap = torch.sort(time_steps_extrap)[0]
print(time_steps_extrap.shape)

tensor([3.7851, 3.3443, 0.1417, 1.3734, 3.0917, 0.3541, 2.0763, 0.4173, 3.6215,
        3.9683, 3.6055, 3.8517, 1.5262, 1.1137, 3.9579, 1.0563, 3.9331, 4.4630,
        3.0525, 4.4004, 2.8379, 1.0336, 0.1935, 2.1825, 0.0623, 0.8038, 0.1491,
        2.8308, 4.4699, 4.7802, 0.0876, 2.9323, 0.6113, 4.8144, 0.1380, 0.2273,
        0.5352, 1.8769, 0.4140, 0.9900, 1.5855, 2.3460, 0.2538, 1.5439, 4.2902,
        3.2616, 2.5595, 4.6672, 3.3973, 2.9913, 1.8209, 4.3530, 3.2576, 3.6419,
        2.3678, 1.2531, 1.5027, 0.8094, 2.2725, 3.1943, 0.9322, 0.1816, 3.5082,
        1.3179, 3.5005, 3.7536, 4.8195, 0.2850, 4.3183, 3.9178, 1.1724, 2.6610,
        1.8388, 0.0661, 2.3603, 1.7746, 4.1492, 0.1078, 4.0930, 2.2240, 2.0680,
        1.2015, 3.2396, 1.0108, 2.0369, 0.2006, 1.9468, 0.9870, 0.1718, 2.8426,
        3.3702, 4.5436, 1.2397, 1.8489, 4.0469, 2.2679, 2.1030, 1.8386, 1.4328])
torch.Size([99])
torch.Size([100])
torch.Size([100])


# Run Model of the author

In [ ]:
###########################
# Latent ODEs for Irregularly-Sampled Time Series
# Author: Yulia Rubanova
###########################

import os
import sys
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot
import matplotlib.pyplot as plt

import time
import datetime
import argparse
import numpy as np
import pandas as pd
from random import SystemRandom
from sklearn import model_selection

import torch
import torch.nn as nn
from torch.nn.functional import relu
import torch.optim as optim

import lib.utils as utils
from lib.plotting import *

from lib.rnn_baselines import *
from lib.ode_rnn import *
from lib.create_latent_ode_model import create_LatentODE_model
from lib.parse_datasets import parse_datasets
from lib.ode_func import ODEFunc, ODEFunc_w_Poisson
from lib.diffeq_solver import DiffeqSolver
from mujoco_physics import HopperPhysics

from lib.utils import compute_loss_all_batches

# Generative model for noisy data based on ODE
parser = argparse.ArgumentParser('Latent ODE')
parser.add_argument('-n',  type=int, default=100, help="Size of the dataset")
parser.add_argument('--niters', type=int, default=300)
parser.add_argument('--lr',  type=float, default=1e-2, help="Starting learning rate.")
parser.add_argument('-b', '--batch-size', type=int, default=50)
parser.add_argument('--viz', action='store_true', help="Show plots while training")

parser.add_argument('--save', type=str, default='experiments/', help="Path for save checkpoints")
parser.add_argument('--load', type=str, default=None, help="ID of the experiment to load for evaluation. If None, run a new experiment.")
parser.add_argument('-r', '--random-seed', type=int, default=1991, help="Random_seed")

parser.add_argument('--dataset', type=str, default='periodic', help="Dataset to load. Available: physionet, activity, hopper, periodic")
parser.add_argument('-s', '--sample-tp', type=float, default=None, help="Number of time points to sub-sample."
	"If > 1, subsample exact number of points. If the number is in [0,1], take a percentage of available points per time series. If None, do not subsample")

parser.add_argument('-c', '--cut-tp', type=int, default=None, help="Cut out the section of the timeline of the specified length (in number of points)."
	"Used for periodic function demo.")

parser.add_argument('--quantization', type=float, default=0.1, help="Quantization on the physionet dataset."
	"Value 1 means quantization by 1 hour, value 0.1 means quantization by 0.1 hour = 6 min")

parser.add_argument('--latent-ode', action='store_true', help="Run Latent ODE seq2seq model")
parser.add_argument('--z0-encoder', type=str, default='odernn', help="Type of encoder for Latent ODE model: odernn or rnn")

parser.add_argument('--classic-rnn', action='store_true', help="Run RNN baseline: classic RNN that sees true points at every point. Used for interpolation only.")
parser.add_argument('--rnn-cell', default="gru", help="RNN Cell type. Available: gru (default), expdecay")
parser.add_argument('--input-decay', action='store_true', help="For RNN: use the input that is the weighted average of impirical mean and previous value (like in GRU-D)")

parser.add_argument('--ode-rnn', action='store_true', help="Run ODE-RNN baseline: RNN-style that sees true points at every point. Used for interpolation only.")

parser.add_argument('--rnn-vae', action='store_true', help="Run RNN baseline: seq2seq model with sampling of the h0 and ELBO loss.")

parser.add_argument('-l', '--latents', type=int, default=6, help="Size of the latent state")
parser.add_argument('--rec-dims', type=int, default=20, help="Dimensionality of the recognition model (ODE or RNN).")

parser.add_argument('--rec-layers', type=int, default=1, help="Number of layers in ODE func in recognition ODE")
parser.add_argument('--gen-layers', type=int, default=1, help="Number of layers in ODE func in generative ODE")

parser.add_argument('-u', '--units', type=int, default=100, help="Number of units per layer in ODE func")
parser.add_argument('-g', '--gru-units', type=int, default=100, help="Number of units per layer in each of GRU update networks")

parser.add_argument('--poisson', action='store_true', help="Model poisson-process likelihood for the density of events in addition to reconstruction.")
parser.add_argument('--classif', action='store_true', help="Include binary classification loss -- used for Physionet dataset for hospiral mortality")

parser.add_argument('--linear-classif', action='store_true', help="If using a classifier, use a linear classifier instead of 1-layer NN")
parser.add_argument('--extrap', action='store_true', help="Set extrapolation mode. If this flag is not set, run interpolation mode.")

parser.add_argument('-t', '--timepoints', type=int, default=100, help="Total number of time-points")
parser.add_argument('--max-t',  type=float, default=5., help="We subsample points in the interval [0, args.max_tp]")
parser.add_argument('--noise-weight', type=float, default=0.01, help="Noise amplitude for generated traejctories")


args = parser.parse_args()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
file_name = os.path.basename(__file__)[:-3]
utils.makedirs(args.save)

#####################################################################################################

if __name__ == '__main__':
	torch.manual_seed(args.random_seed)
	np.random.seed(args.random_seed)

	experimentID = args.load
	if experimentID is None:
		# Make a new experiment ID
		experimentID = int(SystemRandom().random()*100000)
	ckpt_path = os.path.join(args.save, "experiment_" + str(experimentID) + '.ckpt')

	start = time.time()
	print("Sampling dataset of {} training examples".format(args.n))
	
	input_command = sys.argv
	ind = [i for i in range(len(input_command)) if input_command[i] == "--load"]
	if len(ind) == 1:
		ind = ind[0]
		input_command = input_command[:ind] + input_command[(ind+2):]
	input_command = " ".join(input_command)

	utils.makedirs("results/")

	##################################################################
	data_obj = parse_datasets(args, device)
	input_dim = data_obj["input_dim"]

	classif_per_tp = False
	if ("classif_per_tp" in data_obj):
		# do classification per time point rather than on a time series as a whole
		classif_per_tp = data_obj["classif_per_tp"]

	if args.classif and (args.dataset == "hopper" or args.dataset == "periodic"):
		raise Exception("Classification task is not available for MuJoCo and 1d datasets")

	n_labels = 1
	if args.classif:
		if ("n_labels" in data_obj):
			n_labels = data_obj["n_labels"]
		else:
			raise Exception("Please provide number of labels for classification task")

	##################################################################
	# Create the model
	obsrv_std = 0.01
	if args.dataset == "hopper":
		obsrv_std = 1e-3 

	obsrv_std = torch.Tensor([obsrv_std]).to(device)

	z0_prior = Normal(torch.Tensor([0.0]).to(device), torch.Tensor([1.]).to(device))

	if args.rnn_vae:
		if args.poisson:
			print("Poisson process likelihood not implemented for RNN-VAE: ignoring --poisson")

		# Create RNN-VAE model
		model = RNN_VAE(input_dim, args.latents, 
			device = device, 
			rec_dims = args.rec_dims, 
			concat_mask = True, 
			obsrv_std = obsrv_std,
			z0_prior = z0_prior,
			use_binary_classif = args.classif,
			classif_per_tp = classif_per_tp,
			linear_classifier = args.linear_classif,
			n_units = args.units,
			input_space_decay = args.input_decay,
			cell = args.rnn_cell,
			n_labels = n_labels,
			train_classif_w_reconstr = (args.dataset == "physionet")
			).to(device)


	elif args.classic_rnn:
		if args.poisson:
			print("Poisson process likelihood not implemented for RNN: ignoring --poisson")

		if args.extrap:
			raise Exception("Extrapolation for standard RNN not implemented")
		# Create RNN model
		model = Classic_RNN(input_dim, args.latents, device, 
			concat_mask = True, obsrv_std = obsrv_std,
			n_units = args.units,
			use_binary_classif = args.classif,
			classif_per_tp = classif_per_tp,
			linear_classifier = args.linear_classif,
			input_space_decay = args.input_decay,
			cell = args.rnn_cell,
			n_labels = n_labels,
			train_classif_w_reconstr = (args.dataset == "physionet")
			).to(device)
	elif args.ode_rnn:
		# Create ODE-GRU model
		n_ode_gru_dims = args.latents
				
		if args.poisson:
			print("Poisson process likelihood not implemented for ODE-RNN: ignoring --poisson")

		if args.extrap:
			raise Exception("Extrapolation for ODE-RNN not implemented")

		ode_func_net = utils.create_net(n_ode_gru_dims, n_ode_gru_dims, 
			n_layers = args.rec_layers, n_units = args.units, nonlinear = nn.Tanh)

		rec_ode_func = ODEFunc(
			input_dim = input_dim, 
			latent_dim = n_ode_gru_dims,
			ode_func_net = ode_func_net,
			device = device).to(device)

		z0_diffeq_solver = DiffeqSolver(input_dim, rec_ode_func, "euler", args.latents, 
			odeint_rtol = 1e-3, odeint_atol = 1e-4, device = device)
	
		model = ODE_RNN(input_dim, n_ode_gru_dims, device = device, 
			z0_diffeq_solver = z0_diffeq_solver, n_gru_units = args.gru_units,
			concat_mask = True, obsrv_std = obsrv_std,
			use_binary_classif = args.classif,
			classif_per_tp = classif_per_tp,
			n_labels = n_labels,
			train_classif_w_reconstr = (args.dataset == "physionet")
			).to(device)
	elif args.latent_ode:
		model = create_LatentODE_model(args, input_dim, z0_prior, obsrv_std, device, 
			classif_per_tp = classif_per_tp,
			n_labels = n_labels)
	else:
		raise Exception("Model not specified")

	##################################################################

	if args.viz:
		viz = Visualizations(device)

	##################################################################
	
	#Load checkpoint and evaluate the model
	if args.load is not None:
		utils.get_ckpt_model(ckpt_path, model, device)
		exit()

	##################################################################
	# Training

	log_path = "logs/" + file_name + "_" + str(experimentID) + ".log"
	if not os.path.exists("logs/"):
		utils.makedirs("logs/")
	logger = utils.get_logger(logpath=log_path, filepath=os.path.abspath(__file__))
	logger.info(input_command)

	optimizer = optim.Adamax(model.parameters(), lr=args.lr)

	num_batches = data_obj["n_train_batches"]

	for itr in range(1, num_batches * (args.niters + 1)):
		optimizer.zero_grad()
		utils.update_learning_rate(optimizer, decay_rate = 0.999, lowest = args.lr / 10)

		wait_until_kl_inc = 10
		if itr // num_batches < wait_until_kl_inc:
			kl_coef = 0.
		else:
			kl_coef = (1-0.99** (itr // num_batches - wait_until_kl_inc))

		batch_dict = utils.get_next_batch(data_obj["train_dataloader"])
		train_res = model.compute_all_losses(batch_dict, n_traj_samples = 3, kl_coef = kl_coef)
		train_res["loss"].backward()
		optimizer.step()

		n_iters_to_viz = 1
		if itr % (n_iters_to_viz * num_batches) == 0:
			with torch.no_grad():

				test_res = compute_loss_all_batches(model, 
					data_obj["test_dataloader"], args,
					n_batches = data_obj["n_test_batches"],
					experimentID = experimentID,
					device = device,
					n_traj_samples = 3, kl_coef = kl_coef)

				message = 'Epoch {:04d} [Test seq (cond on sampled tp)] | Loss {:.6f} | Likelihood {:.6f} | KL fp {:.4f} | FP STD {:.4f}|'.format(
					itr//num_batches, 
					test_res["loss"].detach(), test_res["likelihood"].detach(), 
					test_res["kl_first_p"], test_res["std_first_p"])
		 	
				logger.info("Experiment " + str(experimentID))
				logger.info(message)
				logger.info("KL coef: {}".format(kl_coef))
				logger.info("Train loss (one batch): {}".format(train_res["loss"].detach()))
				logger.info("Train CE loss (one batch): {}".format(train_res["ce_loss"].detach()))
				
				if "auc" in test_res:
					logger.info("Classification AUC (TEST): {:.4f}".format(test_res["auc"]))

				if "mse" in test_res:
					logger.info("Test MSE: {:.4f}".format(test_res["mse"]))

				if "accuracy" in train_res:
					logger.info("Classification accuracy (TRAIN): {:.4f}".format(train_res["accuracy"]))

				if "accuracy" in test_res:
					logger.info("Classification accuracy (TEST): {:.4f}".format(test_res["accuracy"]))

				if "pois_likelihood" in test_res:
					logger.info("Poisson likelihood: {}".format(test_res["pois_likelihood"]))

				if "ce_loss" in test_res:
					logger.info("CE loss: {}".format(test_res["ce_loss"]))

			torch.save({
				'args': args,
				'state_dict': model.state_dict(),
			}, ckpt_path)


			# Plotting
			if args.viz:
				with torch.no_grad():
					test_dict = utils.get_next_batch(data_obj["test_dataloader"])

					print("plotting....")
					if isinstance(model, LatentODE) and (args.dataset == "periodic"): #and not args.classic_rnn and not args.ode_rnn:
						plot_id = itr // num_batches // n_iters_to_viz
						viz.draw_all_plots_one_dim(test_dict, model, 
							plot_name = file_name + "_" + str(experimentID) + "_{:03d}".format(plot_id) + ".png",
						 	experimentID = experimentID, save=True)
						plt.pause(0.01)
	torch.save({
		'args': args,
		'state_dict': model.state_dict(),
	}, ckpt_path)



usage: Latent ODE [-h] [-n N] [--niters NITERS] [--lr LR] [-b BATCH_SIZE]
                  [--viz] [--save SAVE] [--load LOAD] [-r RANDOM_SEED]
                  [--dataset DATASET] [-s SAMPLE_TP] [-c CUT_TP]
                  [--quantization QUANTIZATION] [--latent-ode]
                  [--z0-encoder Z0_ENCODER] [--classic-rnn]
                  [--rnn-cell RNN_CELL] [--input-decay] [--ode-rnn]
                  [--rnn-vae] [-l LATENTS] [--rec-dims REC_DIMS]
                  [--rec-layers REC_LAYERS] [--gen-layers GEN_LAYERS]
                  [-u UNITS] [-g GRU_UNITS] [--poisson] [--classif]
                  [--linear-classif] [--extrap] [-t TIMEPOINTS]
                  [--max-t MAX_T] [--noise-weight NOISE_WEIGHT]
Latent ODE: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-b3795be3-3477-4bee-97ed-b38edd553d61.json


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Break it down

In [2]:
# Install the latest version of author's repo neural ode implementation
!git clone https://github.com/rtqichen/torchdiffeq.git
!cd torchdiffeq && pip install -e .
!ls torchdiffeq/torchdiffeq

fatal: destination path 'torchdiffeq' already exists and is not an empty directory.
Obtaining file:///content/torchdiffeq
  Attempting uninstall: torchdiffeq
    Found existing installation: torchdiffeq 0.2.2
    Can't uninstall 'torchdiffeq'. No files were found to uninstall.
  Running setup.py develop for torchdiffeq
_impl  __init__.py


## Libraries

In [55]:
# run_models.py
import os
import sys
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot
import matplotlib.pyplot as plt

import time
import datetime
import argparse
import numpy as np
import pandas as pd
from random import SystemRandom
from sklearn import model_selection

import torch
import torch.nn as nn
from torch.nn.functional import relu
import torch.optim as optim

import lib.utils as utils
from lib.plotting import *

from lib.rnn_baselines import *
from lib.ode_rnn import *
from lib.create_latent_ode_model import create_LatentODE_model
from lib.parse_datasets import parse_datasets
from lib.ode_func import ODEFunc, ODEFunc_w_Poisson
from lib.diffeq_solver import DiffeqSolver
from mujoco_physics import HopperPhysics

from lib.utils import compute_loss_all_batches

import sys
# print(sys.argv[1:])

## Parameters

In [56]:
# Generative model for noisy data based on ODE
parser = argparse.ArgumentParser('Latent ODE')
parser.add_argument('-n',  type=int, default=100, help="Size of the dataset")
parser.add_argument('--niters', type=int, default=300)
parser.add_argument('--lr',  type=float, default=1e-2, help="Starting learning rate.")
parser.add_argument('-b', '--batch-size', type=int, default=50)
parser.add_argument('--viz', action='store_true', help="Show plots while training")

parser.add_argument('--save', type=str, default='experiments/', help="Path for save checkpoints")
parser.add_argument('--load', type=str, default=None, help="ID of the experiment to load for evaluation. If None, run a new experiment.")
parser.add_argument('-r', '--random-seed', type=int, default=1991, help="Random_seed")

parser.add_argument('--dataset', type=str, default='hopper', help="Dataset to load. Available: physionet, activity, hopper, periodic")
parser.add_argument('-s', '--sample-tp', type=float, default=None, help="Number of time points to sub-sample."
	"If > 1, subsample exact number of points. If the number is in [0,1], take a percentage of available points per time series. If None, do not subsample")

parser.add_argument('-c', '--cut-tp', type=int, default=None, help="Cut out the section of the timeline of the specified length (in number of points)."
	"Used for periodic function demo.")

parser.add_argument('--quantization', type=float, default=0.1, help="Quantization on the physionet dataset."
	"Value 1 means quantization by 1 hour, value 0.1 means quantization by 0.1 hour = 6 min")

parser.add_argument('--latent-ode', default = True, action='store_true', help="Run Latent ODE seq2seq model")
parser.add_argument('--z0-encoder', type=str, default='odernn', help="Type of encoder for Latent ODE model: odernn or rnn")

parser.add_argument('--classic-rnn', action='store_true', help="Run RNN baseline: classic RNN that sees true points at every point. Used for interpolation only.")
parser.add_argument('--rnn-cell', default="gru", help="RNN Cell type. Available: gru (default), expdecay")
parser.add_argument('--input-decay', action='store_true', help="For RNN: use the input that is the weighted average of impirical mean and previous value (like in GRU-D)")

parser.add_argument('--ode-rnn', action='store_true', help="Run ODE-RNN baseline: RNN-style that sees true points at every point. Used for interpolation only.")

parser.add_argument('--rnn-vae', action='store_true', help="Run RNN baseline: seq2seq model with sampling of the h0 and ELBO loss.")

parser.add_argument('-l', '--latents', type=int, default=6, help="Size of the latent state")
parser.add_argument('--rec-dims', type=int, default=20, help="Dimensionality of the recognition model (ODE or RNN).")

parser.add_argument('--rec-layers', type=int, default=1, help="Number of layers in ODE func in recognition ODE")
parser.add_argument('--gen-layers', type=int, default=1, help="Number of layers in ODE func in generative ODE")

parser.add_argument('-u', '--units', type=int, default=100, help="Number of units per layer in ODE func")
parser.add_argument('-g', '--gru-units', type=int, default=100, help="Number of units per layer in each of GRU update networks")

parser.add_argument('--poisson', action='store_true', help="Model poisson-process likelihood for the density of events in addition to reconstruction.")
parser.add_argument('--classif', action='store_true', help="Include binary classification loss -- used for Physionet dataset for hospiral mortality")

parser.add_argument('--linear-classif', action='store_true', help="If using a classifier, use a linear classifier instead of 1-layer NN")
parser.add_argument('--extrap', action='store_true', help="Set extrapolation mode. If this flag is not set, run interpolation mode.")

parser.add_argument('-t', '--timepoints', type=int, default=100, help="Total number of time-points")
parser.add_argument('--max-t',  type=float, default=5., help="We subsample points in the interval [0, args.max_tp]")
parser.add_argument('--noise-weight', type=float, default=0.01, help="Noise amplitude for generated traejctories")

sys.argv = ['-f']
args = parser.parse_args()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# file_name = os.path.basename(__file__)[:-3]
utils.makedirs(args.save)  # saves in 'experiments/' folder

## Manual seed, experimentID

In [21]:
torch.manual_seed(args.random_seed)
np.random.seed(args.random_seed)

experimentID = args.load  # None
print(f"experimentID is {experimentID}")

experimentID is None


In [22]:
if experimentID is None:
		# Make a new experiment ID
		experimentID = int(SystemRandom().random()*100000) # from random import SystemRandom
print(f"experimentID is {experimentID}")
ckpt_path = os.path.join(args.save, "experiment_" + str(experimentID) + '.ckpt') 
print(f"ckpt_path is {ckpt_path}")

experimentID is 70226
ckpt_path is experiments/experiment_70226.ckpt


In [23]:
start = time.time()
print("Sampling dataset of {} training examples".format(args.n))  # n is size of the dataset

Sampling dataset of 100 training examples


In [24]:
print(f"type of args is {type(args)}")
print(f"args is {str(args)}")

type of args is <class 'argparse.Namespace'>
args is Namespace(batch_size=50, classic_rnn=False, classif=False, cut_tp=None, dataset='hopper', extrap=False, gen_layers=1, gru_units=100, input_decay=False, latent_ode=True, latents=6, linear_classif=False, load=None, lr=0.01, max_t=5.0, n=100, niters=300, noise_weight=0.01, ode_rnn=False, poisson=False, quantization=0.1, random_seed=1991, rec_dims=20, rec_layers=1, rnn_cell='gru', rnn_vae=False, sample_tp=None, save='experiments/', timepoints=100, units=100, viz=False, z0_encoder='odernn')


In [25]:
input_command = sys.argv
print(f"input_command is {input_command}")


input_command is ['-f']


In [26]:
ind = [i for i in range(len(input_command)) if input_command[i] == "--load"]
print(f"ind is {ind}")
print(f"len(ind) is {len(ind)}")

ind is []
len(ind) is 0


In [27]:
if len(ind) == 1:
		ind = ind[0]
		input_command = input_command[:ind] + input_command[(ind+2):]
input_command = " ".join(input_command)
print(f"input_command is {input_command}")
utils.makedirs("results/")


input_command is -f


## parse_datasets()

In [28]:
data_obj = parse_datasets(args, device)


Using the function parse_datasets from parse_datasets.py
Inputs are (args, device)
dataset_name is hopper


args.timepoints is 100
args.extrap is False
n_total_tp = args.timepoints + args.extrap  --> is 100


args.max_t is 5.0
max_t_extrap = args.max_t / args.timepoints * n_total_tp --> 5.0


if dataset_name == 'hopper'
root is data
Let's download
we will check os.path.exists(os.path.join(self.data_folder, self.training_file)). If it exists, return
we will check os.path.exists(os.path.join(self.data_folder, self.training_file)). If it exists, return
data_file is data/HopperPhysics/training.pt
Using torch.Tensor(torch.load(data_file)).to(device) to get self.data
self.data.shape is torch.Size([10000, 200, 14])


Using normalize_data()
data.shape is torch.Size([10000, 200, 14])
reshaped = data.reshape(-1, data.size(-1))
reshaped.shape is torch.Size([2000000, 14])
att_min = torch.min(reshaped, 0)[0]
att_min.shape is torch.Size([14])
att_max = torch.max(reshaped, 0)[0]
att_max.shape is torc

In [29]:
input_dim = data_obj["input_dim"]
n_train_batches = data_obj['n_train_batches']
n_test_batches = data_obj['n_test_batches']
print(f"input_dim is {input_dim}")
print(f"n_train_batches is {n_train_batches}")
print(f"n_test_batches is {n_test_batches}")

input_dim is 14
n_train_batches is 2
n_test_batches is 1


In [30]:
data_obj.keys()

dict_keys(['dataset_obj', 'train_dataloader', 'test_dataloader', 'input_dim', 'n_train_batches', 'n_test_batches'])

## Create Model

In [31]:
classif_per_tp = False
if ("classif_per_tp" in data_obj):
  # do classification per time point rather than on a time series as a whole
  classif_per_tp = data_obj["classif_per_tp"]

if args.classif and (args.dataset == "hopper" or args.dataset == "periodic"):
  raise Exception("Classification task is not available for MuJoCo and 1d datasets")

n_labels = 1
if args.classif:
  if ("n_labels" in data_obj):
    n_labels = data_obj["n_labels"]
  else:
    raise Exception("Please provide number of labels for classification task")


In [32]:
print(f"n_labels is {n_labels}")

n_labels is 1


In [33]:
obsrv_std = 0.01
if args.dataset == "hopper":
  obsrv_std = 1e-3 

obsrv_std = torch.Tensor([obsrv_std]).to(device)
print(f"obsrv_std is {obsrv_std}")
z0_prior = Normal(torch.Tensor([0.0]).to(device), torch.Tensor([1.]).to(device))
print(f"z0_prior is {z0_prior}")

obsrv_std is tensor([0.0010])
z0_prior is Normal(loc: tensor([0.]), scale: tensor([1.]))


In [42]:
if args.latent_ode:
    model = create_LatentODE_model(args, input_dim, z0_prior, obsrv_std, device, 
            classif_per_tp = classif_per_tp,
            n_labels = n_labels)
else:
	raise Exception("Model not specified")

Inside create_LatentODE_model
dim is 6


Making ode_func_net
Inside create_net function
n_input for layers is 6
n_units for layers is 100
n_layers is 1
nonlinear is <class 'torch.nn.modules.activation.Tanh'>
for i in range(n_layers):
    layers.append(nonlinear())
    layers.append(nn.Linear(n_units, n_units))
layers.append(nonlinear())
layers.append(nn.Linear(n_units, n_outputs))
n_outputs is 6
Making gen_ode_func
Inside ODEFunc class
input_dim is 14
latent_dim is 6
Inside init_network_weights function
std is 0.1
Making ode_func_net for odernn
Inside create_net function
n_input for layers is 20
n_units for layers is 100
n_layers is 1
nonlinear is <class 'torch.nn.modules.activation.Tanh'>
for i in range(n_layers):
    layers.append(nonlinear())
    layers.append(nn.Linear(n_units, n_units))
layers.append(nonlinear())
layers.append(nn.Linear(n_units, n_outputs))
n_outputs is 20
Making rec_ode_func using ODEFunc
Inside ODEFunc class
input_dim is 28
latent_dim is 20
Inside init_network_w

In [39]:
t1 = torch.tensor([1,2,3,4]) 
t2 = torch.tensor([5,6,7,8]) 
t3 = torch.tensor([9,10,11,12])

t = torch.stack(
    (t1,t2,t3)
    ,dim=-1
)

In [57]:
utils.split_last_dim(t)

(tensor([[1],
         [2],
         [3],
         [4]]), tensor([[ 5,  9],
         [ 6, 10],
         [ 7, 11],
         [ 8, 12]]))

In [53]:
from lib import utils as utils

In [54]:
utils.split_last_dim(t)

(tensor([[1],
         [2],
         [3],
         [4]]), tensor([[ 5,  9],
         [ 6, 10],
         [ 7, 11],
         [ 8, 12]]))

# For Stock Data

In [58]:
# Install the latest version of author's repo neural ode implementation
!git clone https://github.com/rtqichen/torchdiffeq.git
!cd torchdiffeq && pip install -e .
!ls torchdiffeq/torchdiffeq



fatal: destination path 'torchdiffeq' already exists and is not an empty directory.
Obtaining file:///content/torchdiffeq
  Attempting uninstall: torchdiffeq
    Found existing installation: torchdiffeq 0.2.2
    Can't uninstall 'torchdiffeq'. No files were found to uninstall.
  Running setup.py develop for torchdiffeq
_impl  __init__.py  __pycache__


## Libraries

In [59]:
# run_models.py
import os
import sys
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot
import matplotlib.pyplot as plt

import time
import datetime
import argparse
import numpy as np
import pandas as pd
from random import SystemRandom
from sklearn import model_selection

import torch
import torch.nn as nn
from torch.nn.functional import relu
import torch.optim as optim

import lib.utils as utils
from lib.plotting import *

from lib.rnn_baselines import *
from lib.ode_rnn import *
from lib.create_latent_ode_model import create_LatentODE_model
from lib.parse_datasets import parse_datasets
from lib.ode_func import ODEFunc, ODEFunc_w_Poisson
from lib.diffeq_solver import DiffeqSolver
from mujoco_physics import HopperPhysics

from lib.utils import compute_loss_all_batches

import sys
# print(sys.argv[1:])

## Parameters

In [60]:
# Generative model for noisy data based on ODE
parser = argparse.ArgumentParser('Latent ODE')
parser.add_argument('-n',  type=int, default=100, help="Size of the dataset")
parser.add_argument('--niters', type=int, default=300)
parser.add_argument('--lr',  type=float, default=1e-2, help="Starting learning rate.")
parser.add_argument('-b', '--batch-size', type=int, default=50)
parser.add_argument('--viz', action='store_true', help="Show plots while training")

parser.add_argument('--save', type=str, default='experiments/', help="Path for save checkpoints")
parser.add_argument('--load', type=str, default=None, help="ID of the experiment to load for evaluation. If None, run a new experiment.")
parser.add_argument('-r', '--random-seed', type=int, default=1991, help="Random_seed")

parser.add_argument('--dataset', type=str, default='hopper', help="Dataset to load. Available: physionet, activity, hopper, periodic")
parser.add_argument('-s', '--sample-tp', type=float, default=None, help="Number of time points to sub-sample."
	"If > 1, subsample exact number of points. If the number is in [0,1], take a percentage of available points per time series. If None, do not subsample")

parser.add_argument('-c', '--cut-tp', type=int, default=None, help="Cut out the section of the timeline of the specified length (in number of points)."
	"Used for periodic function demo.")

parser.add_argument('--quantization', type=float, default=0.1, help="Quantization on the physionet dataset."
	"Value 1 means quantization by 1 hour, value 0.1 means quantization by 0.1 hour = 6 min")

parser.add_argument('--latent-ode', default = True, action='store_true', help="Run Latent ODE seq2seq model")
parser.add_argument('--z0-encoder', type=str, default='odernn', help="Type of encoder for Latent ODE model: odernn or rnn")

parser.add_argument('--classic-rnn', action='store_true', help="Run RNN baseline: classic RNN that sees true points at every point. Used for interpolation only.")
parser.add_argument('--rnn-cell', default="gru", help="RNN Cell type. Available: gru (default), expdecay")
parser.add_argument('--input-decay', action='store_true', help="For RNN: use the input that is the weighted average of impirical mean and previous value (like in GRU-D)")

parser.add_argument('--ode-rnn', action='store_true', help="Run ODE-RNN baseline: RNN-style that sees true points at every point. Used for interpolation only.")

parser.add_argument('--rnn-vae', action='store_true', help="Run RNN baseline: seq2seq model with sampling of the h0 and ELBO loss.")

parser.add_argument('-l', '--latents', type=int, default=6, help="Size of the latent state")
parser.add_argument('--rec-dims', type=int, default=20, help="Dimensionality of the recognition model (ODE or RNN).")

parser.add_argument('--rec-layers', type=int, default=1, help="Number of layers in ODE func in recognition ODE")
parser.add_argument('--gen-layers', type=int, default=1, help="Number of layers in ODE func in generative ODE")

parser.add_argument('-u', '--units', type=int, default=100, help="Number of units per layer in ODE func")
parser.add_argument('-g', '--gru-units', type=int, default=100, help="Number of units per layer in each of GRU update networks")

parser.add_argument('--poisson', action='store_true', help="Model poisson-process likelihood for the density of events in addition to reconstruction.")
parser.add_argument('--classif', action='store_true', help="Include binary classification loss -- used for Physionet dataset for hospiral mortality")

parser.add_argument('--linear-classif', action='store_true', help="If using a classifier, use a linear classifier instead of 1-layer NN")
parser.add_argument('--extrap', action='store_true', help="Set extrapolation mode. If this flag is not set, run interpolation mode.")

parser.add_argument('-t', '--timepoints', type=int, default=100, help="Total number of time-points")
parser.add_argument('--max-t',  type=float, default=5., help="We subsample points in the interval [0, args.max_tp]")
parser.add_argument('--noise-weight', type=float, default=0.01, help="Noise amplitude for generated traejctories")

sys.argv = ['-f']
args = parser.parse_args()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# file_name = os.path.basename(__file__)[:-3]
utils.makedirs(args.save)  # saves in 'experiments/' folder

## Manual seed, experimentID

In [61]:
torch.manual_seed(args.random_seed)
np.random.seed(args.random_seed)

experimentID = args.load  # None
print(f"experimentID is {experimentID}")

if experimentID is None:
		# Make a new experiment ID
		experimentID = int(SystemRandom().random()*100000) # from random import SystemRandom
print(f"experimentID is {experimentID}")
ckpt_path = os.path.join(args.save, "experiment_" + str(experimentID) + '.ckpt') 
print(f"ckpt_path is {ckpt_path}")

start = time.time()
print("Sampling dataset of {} training examples".format(args.n))  # n is size of the dataset

print(f"args is {str(args)}")

input_command = sys.argv
print(f"input_command is {input_command}")

ind = [i for i in range(len(input_command)) if input_command[i] == "--load"]
print(f"ind is {ind}")
print(f"len(ind) is {len(ind)}")

if len(ind) == 1:
		ind = ind[0]
		input_command = input_command[:ind] + input_command[(ind+2):]
input_command = " ".join(input_command)
print(f"input_command is {input_command}")
utils.makedirs("results/")


experimentID is None
experimentID is 13863
ckpt_path is experiments/experiment_13863.ckpt
Sampling dataset of 100 training examples
args is Namespace(batch_size=50, classic_rnn=False, classif=False, cut_tp=None, dataset='hopper', extrap=False, gen_layers=1, gru_units=100, input_decay=False, latent_ode=True, latents=6, linear_classif=False, load=None, lr=0.01, max_t=5.0, n=100, niters=300, noise_weight=0.01, ode_rnn=False, poisson=False, quantization=0.1, random_seed=1991, rec_dims=20, rec_layers=1, rnn_cell='gru', rnn_vae=False, sample_tp=None, save='experiments/', timepoints=100, units=100, viz=False, z0_encoder='odernn')
input_command is ['-f']
ind is []
len(ind) is 0
input_command is -f


## Get Dataset

In [62]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 3.6 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=4065379c39b442292edaa407b1da8a9042a909b5238476d9a910eb394f3430b0
  Stored in directory: /root/.cache/pip/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [63]:
# 1. Get Open and Close Price of asset (o, c) for each trading day.
# libraries
from pandas_datareader import data as pdr
import yfinance as yf
import os

print(f"Get Open and Close Price of Assets")
def download_raw_stock_data(filepath, tickers, start, end, period = '1d'):
    """
    Download Stock tickers
    :Parameters:
        filepath: str
            path to store the raw data
        tickers : str, list
            List of tickers to download
        period: str
            the frequency at which to gather the data; common options would include ‘1d’ (daily), ‘1mo’ (monthly), ‘1y’ (yearly)
        start: str
            the date to start gathering the data. For example ‘2010–1–1’
        end: str
            the date to end gathering the data. For example ‘2020–1–25’
    
    """
    #define the ticker symbol
    tickerSymbol = tickers

    #get data on this ticker
    tickerData = yf.Ticker(tickerSymbol)

    #get the historical prices for this ticker
    tickerDf = tickerData.history(period=period, start=start, end=end)
    tickerDf.to_csv(filepath)

dict_tickers = {
    'Apple': 'AAPL',
    'Microsoft': 'MSFT',
    'Google': 'GOOG',
    'Bitcoin': 'BTC-USD',
    'Facebook': 'FB',
    'Walmart': 'WMT',
    'Amazon': 'AMZN',
    'CVS': 'CVS',
    'Berkshire': 'BRK-B',
    'ExxonMobil': 'XOM',
    'AtandT': 'T',
    'Costco': 'COST',
    'Walgreens': 'WBA',
    'Kroger': 'KR',
    'JPMorgan': 'JPM',
    'Verizon': 'VZ',
    'FordMotor': 'F',
    'GeneralMotors': 'GM',
    'Dell': 'DELL',
    'BankOfAmerica': 'BAC',
    'Target': 'TGT',
    'GeneralElectric': 'GE',
    'JohnsonandJohnson': 'JNJ',
    'Nvidia': 'NVDA',
    'Intel': 'INTC',
}

path = f"raw-stock-data/data-2000-2021"
if not os.path.exists(path):
    # https://appdividend.com/2021/07/03/how-to-create-directory-if-not-exist-in-python/
    # Create a new directory
    os.makedirs(path)
    print(f"{path} directory is created")
period = '1d'
start='2000-1-1'
end='2021-8-31'
for tickerName, ticker in dict_tickers.items():
    tickerName = tickerName
    ticker = ticker
    filepath = f"{path}/{tickerName}.csv"
    download_raw_stock_data(filepath, ticker, start, end, period)


print('\n')

print(f"The size of each asset")
import pandas as pd
for tickerName in dict_tickers.keys():
    df = pd.read_csv(f"{path}/{tickerName}.csv")
    print(f"{tickerName} size: {len(df)}")

# 2. Get weekly data.
# 3. Transform $d_{i}$ to sequences of lag * len($d_{i}$) length.

def stockDataTransformer(filepath):
    df = pd.read_csv(filepath)
    df.set_index('Date', inplace=True)
    df1 = df[['Open', 'Close']].copy()
    data = df1.values
    n_samples = data.shape[0]//10*10
    reshape_number = n_samples*data.shape[1]//10
    data1 = data[:n_samples].reshape((reshape_number, 10))
    return data1

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def split_data(perc_train, perc_valid, lag, data_orig, data_m1, n_features_orig, n_features_median):
    values = data_m1
    
    sizeOfReframed = len(data_m1)
    len_train = int(perc_train*sizeOfReframed) # int(sizeOfReframed - len_test) # - len_valid)
    train_data_orig = data_orig[:len_train, :]
    # valid = values[len_train:len_valid+len_train, :]
    test_data_orig = data_orig[len_train:, :]  # [len_valid+len_train:, :]
    # n_features = n_features
    
    train_data_ml = values[:len_train, :]
    test_data_ml = values[len_train:, :] 
    # split into input and outputs
    n_obs = lag * n_features_orig
    n_obs_median = (lag+forecast) * n_features_median
    train_X, train_y = train_data_orig[:, :n_obs], train_data_ml[:, :n_obs_median]
    test_X, test_y = test_data_orig[:, :n_obs], test_data_ml[:, :n_obs_median]
    # valid_X, valid_y = valid[:, :n_obs], valid[:, -1]
    print(train_X.shape, len(train_X), train_y.shape)
    
    # reshape input to be 3D [samples, features, lag]
    train_X = train_X.reshape((train_X.shape[0], n_features_orig, lag))
    test_X = test_X.reshape((test_X.shape[0], n_features_orig, lag))
    # valid_X = valid_X.reshape((valid_X.shape[0], lag, n_features))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)  # , valid_X.shape, valid_y.shape)
    
    # Get the reconstruction train_y, test_y and extrapolated train_y, test_y
    train_y_recon, train_y_extrapol = train_y[:, :lag], train_y[:, lag:]
    test_y_recon, test_y_extrapol = test_y[:, :lag], test_y[:, lag:]
    dataload = {
        'train_data_orig': train_data_orig,
        'test_data_orig': test_data_orig,
        'train_data_ml': train_data_ml,
        'test_data_ml': test_data_ml,
        # 'valid': valid,
        'train_X': train_X,
        'train_y': train_y,
        'test_X': test_X,
        'test_y': test_y,
        'n_features_orig': n_features_orig,
        'n_features_median': n_features_median,
        'n_obs': n_obs,
        'n_obs_median': n_obs_median,
        # 'valid_X': valid_X,
        # 'valid_y': valid_y,
        'train_y_recon': train_y_recon,
        'train_y_extrapol': train_y_extrapol,
        'test_y_recon': test_y_recon,
        'test_y_extrapol': test_y_extrapol
    }
    
    return dataload

def get_median(array, axis = 1):
    # https://numpy.org/doc/stable/reference/generated/numpy.median.html
    return np.median(array, axis = axis).reshape(data_size, 1)  #, keepdims=True)

from pandas import concat
import numpy as np
week_sequence = {}
median_data_dict = {}
lag = 5
for tickerName in dict_tickers.keys():
    filepath = f"{path}/{tickerName}.csv"
    # Get the data in the required format
    data = stockDataTransformer(filepath)
    # # Total Data Size
    data_size = data.shape[0]
    print(f"{tickerName} data.shape {data.shape}")
    data_orig = series_to_supervised(data, lag).values
    print(f'{tickerName} Data Original after series to supervised on data {data_orig.shape}')
    week_sequence[tickerName] = data_orig

    median_data = get_median(data)
    print(f'{tickerName} Median data')
    # Median data for each week
    print(f"{tickerName} median_data.shape {median_data.shape}")
    # print(pd.DataFrame(median_data, columns = ['median_stockprice_week']).head(10))
    print('\n')

    # Convert median_data to (n_samples, 5) matrix
    data_m1 = series_to_supervised(median_data, lag).values
    median_data_dict[tickerName] = data_m1
    print(f'{tickerName} Median data after series to supervised')
    print(f"{tickerName} data_m1.shape {data_m1.shape}")
    # print(pd.DataFrame(data_m1, columns = [f"week i+{i}" for i in range(1, lag+forecast+1)]))
    print('\n')




Get Open and Close Price of Assets
raw-stock-data/data-2000-2021 directory is created


The size of each asset
Apple size: 5450
Microsoft size: 5450
Google size: 4288
Bitcoin size: 2537
Facebook size: 2336
Walmart size: 5450
Amazon size: 5450
CVS size: 5450
Berkshire size: 5450
ExxonMobil size: 5450
AtandT size: 5450
Costco size: 5450
Walgreens size: 5450
Kroger size: 5450
JPMorgan size: 5450
Verizon size: 5450
FordMotor size: 5450
GeneralMotors size: 2713
Dell size: 1268
BankOfAmerica size: 5450
Target size: 5450
GeneralElectric size: 5450
JohnsonandJohnson size: 5450
Nvidia size: 5451
Intel size: 5450
Apple data.shape (1090, 10)
Apple Data Original after series to supervised on data (1085, 60)
Apple Median data
Apple median_data.shape (1090, 1)


Apple Median data after series to supervised
Apple data_m1.shape (1085, 6)


Microsoft data.shape (1090, 10)
Microsoft Data Original after series to supervised on data (1085, 60)
Microsoft Median data
Microsoft median_data.shape (1090, 1)




In [64]:
import numpy as np
data = week_sequence['Apple']
# 4. Bundle all sequences together
for tickerName in week_sequence.keys():
    if tickerName != 'Apple':
        data1 = week_sequence[tickerName]
        data = np.concatenate((data, data1))
        print(f"data.shape {data.shape}")

import numpy as np
data_m = median_data_dict['Apple']
# 4. Bundle all sequences together
for tickerName in median_data_dict.keys():
    if tickerName != 'Apple':
        data1 = median_data_dict[tickerName]
        data_m = np.concatenate((data_m, data1))
        print(f"data.shape {data_m.shape}")

data_df = pd.DataFrame(data)
data_df.to_csv(f"all_assets_sequences.csv")

data_m_df = pd.DataFrame(data_m)
data_m_df.to_csv(f"median_sequences.csv")

data.shape (2170, 60)
data.shape (3021, 60)
data.shape (3522, 60)
data.shape (3983, 60)
data.shape (5068, 60)
data.shape (6153, 60)
data.shape (7238, 60)
data.shape (8323, 60)
data.shape (9408, 60)
data.shape (10493, 60)
data.shape (11578, 60)
data.shape (12663, 60)
data.shape (13748, 60)
data.shape (14833, 60)
data.shape (15918, 60)
data.shape (17003, 60)
data.shape (17540, 60)
data.shape (17787, 60)
data.shape (18872, 60)
data.shape (19957, 60)
data.shape (21042, 60)
data.shape (22127, 60)
data.shape (23206, 60)
data.shape (24291, 60)
data.shape (2170, 6)
data.shape (3021, 6)
data.shape (3522, 6)
data.shape (3983, 6)
data.shape (5068, 6)
data.shape (6153, 6)
data.shape (7238, 6)
data.shape (8323, 6)
data.shape (9408, 6)
data.shape (10493, 6)
data.shape (11578, 6)
data.shape (12663, 6)
data.shape (13748, 6)
data.shape (14833, 6)
data.shape (15918, 6)
data.shape (17003, 6)
data.shape (17540, 6)
data.shape (17787, 6)
data.shape (18872, 6)
data.shape (19957, 6)
data.shape (21042, 6)
data

## Parse Datasets

### Libraries

In [ ]:
import os
import numpy as np

import torch
import torch.nn as nn

import lib.utils as utils
from lib.diffeq_solver import DiffeqSolver
from generate_timeseries import Periodic_1d
from torch.distributions import uniform

from torch.utils.data import DataLoader
# from mujoco_physics import HopperPhysics

from sklearn import model_selection
import random


### HopperPhysics class for our data

#### Libraries

In [65]:

import os
import numpy as np
import torch
from lib.utils import get_dict_template
import lib.utils as utils
from torchvision.datasets.utils import download_url



#### Class

In [ ]:
###########################
# Latent ODEs for Irregularly-Sampled Time Series
# Authors: Yulia Rubanova and Ricky Chen
###########################

import os
import numpy as np
import torch
from lib.utils import get_dict_template
import lib.utils as utils
from torchvision.datasets.utils import download_url
import pandas as pd

class HopperPhysics(object):

    T = 5
    print(f"We are inside HopperPhysics class")
    print(f"T is {T}")
    D = 10
    print("dim is {D}")

    median_dim = 1
    print("median_dim is {median_dim}")

    # n_training_samples = 10000
    # print(f"n_training_samples is {n_training_samples}")

    training_file = 'training.csv'
    print(f"training_file name is {training_file}")

    def __init__(self, root, trainfile, medianfile, download = True, generate=False, device = torch.device("cpu")):
        print(f"root is {root}")
        self.root = root
        if download:
            print(f"Let's load dataset")
            X = pd.read_csv(os.path.join(root, trainfile))
            y = pd.read_csv(os.path.join(root, medianfile))

        if not self._check_exists():
            raise RuntimeError('Dataset not found.' + ' You can use download=True to download it')

        data_file = os.path.join(self.data_folder, self.training_file)
        median_file = os.path.join(self.data_folder, 'median_sequences.csv')
        print(f"data_file is {data_file}")
        # print(f"Using torch.Tensor(torch.load(data_file)).to(device) to get self.data")
        self.data = torch.Tensor(X.values).to(device) # https://pytorch.org/docs/stable/generated/torch.load.html
        self.median_data = torch.Tensor(y.values).to(device)

        self.data = torch.reshape(self.data, (self.data.shape[0], T, D))
        self.median_data = torch.reshape(self.median_data, (self.median_data.shape[0], T, D))
        

        print(f"self.data.shape is {self.data.shape}")
        print(f"self.median_data.shape is {self.median_data.shape}")
        print("\n")
        self.data, self.data_min, self.data_max = utils.normalize_data(self.data)
        self.median_data, self.median_data_min, self.median_data_max = utils.normalize_data(self.median_data)
        self.dataset = list(zip(self.data, self.median_data))

        self.device =device
        self.n_training_samples = self.data.shape[0]
        print(f"self.n_training_samples is {self.n_training_samples}")

    def _check_exists(self):
        print(f"we will check os.path.exists(os.path.join(self.data_folder, self.training_file)). If it exists, return")
        return os.path.exists(os.path.join(self.data_folder, self.training_file))

    @property
    def data_folder(self):
        return self.root

        # def __getitem__(self, index):
        #     return self.data[index]

    def get_dataset(self):
        return self.dataset

    def __len__(self):
        return len(self.data)

    def size(self, ind = None):
        if ind is not None:
            return self.data.shape[ind]
            return self.data.shape
                
    def __repr__(self):
        fmt_str = 'Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
        fmt_str += '    Root Location: {}\n'.format(self.root)
        return fmt_str



#### parse_datasets

In [3]:
import os
import numpy as np

import torch
import torch.nn as nn

import lib.utils as utils
from lib.diffeq_solver import DiffeqSolver
from generate_timeseries import Periodic_1d
from torch.distributions import uniform

from torch.utils.data import DataLoader

from sklearn import model_selection
import random



ModuleNotFoundError: No module named 'torch'

In [ ]:
def parse_datasets(args, device):

	# batch the data and median_data together and don't shuffle
	def basic_collate_fn(batch, time_steps, args = args, device = device, data_type = "train"):
    	# batch looks like [(x0,y0), (x4,y4), (x2,y2)... ]
		
    	data = torch.stack([item[0] for item in batch])
		median_data = torch.stack([item[1] for item in batch])
		data_dict = {
			"data": data, 
			"median_data": median_data,
			"time_steps": time_steps}
		return data, median_data
		
	

	def basic_collate_fn(batch, time_steps, args = args, device = device, data_type = "train"):
		batch = torch.stack(batch)
		data_dict = {
			"data": batch, 
			"time_steps": time_steps}

		data_dict = utils.split_and_subsample_batch(data_dict, args, data_type = data_type)
		return data_dict


	dataset_name = args.dataset

	n_total_tp = args.timepoints + args.extrap
	max_t_extrap = args.max_t / args.timepoints * n_total_tp

	##################################################################
	# MuJoCo dataset
	if dataset_name == "hopper":
		dataset_obj = HopperPhysics(root='data', download=True, generate=False, device = device)
		dataset = dataset_obj.get_dataset()[:args.n]
		dataset = dataset.to(device)


		n_tp_data = dataset[:].shape[1]

		# Time steps that are used later on for exrapolation
		time_steps = torch.arange(start=0, end = n_tp_data, step=1).float().to(device)
		time_steps = time_steps / len(time_steps)

		dataset = dataset.to(device)
		time_steps = time_steps.to(device)

		if not args.extrap:
			# Creating dataset for interpolation
			# sample time points from different parts of the timeline, 
			# so that the model learns from different parts of hopper trajectory
			n_traj = len(dataset)
			n_tp_data = dataset.shape[1]
			n_reduced_tp = args.timepoints

			# sample time points from different parts of the timeline, 
			# so that the model learns from different parts of hopper trajectory
			start_ind = np.random.randint(0, high=n_tp_data - n_reduced_tp +1, size=n_traj)
			end_ind = start_ind + n_reduced_tp
			sliced = []
			for i in range(n_traj):
				  sliced.append(dataset[i, start_ind[i] : end_ind[i], :])
			dataset = torch.stack(sliced).to(device)
			time_steps = time_steps[:n_reduced_tp]

		# Split into train and test by the time sequences
		train_y, test_y = utils.split_train_test(dataset, train_fraq = 0.8)

		n_samples = len(dataset)
		input_dim = dataset.size(-1)

		batch_size = min(args.batch_size, args.n)
		train_dataloader = DataLoader(train_y, batch_size = batch_size, shuffle=False,
			collate_fn= lambda batch: basic_collate_fn(batch, time_steps, data_type = "train"))
		test_dataloader = DataLoader(test_y, batch_size = n_samples, shuffle=False,
			collate_fn= lambda batch: basic_collate_fn(batch, time_steps, data_type = "test"))
		
		data_objects = {"dataset_obj": dataset_obj, 
					"train_dataloader": utils.inf_generator(train_dataloader), 
					"test_dataloader": utils.inf_generator(test_dataloader),
					"input_dim": input_dim,
					"n_train_batches": len(train_dataloader),
					"n_test_batches": len(test_dataloader)}
		return data_objects

	##################################################################
	# Physionet dataset

	if dataset_name == "physionet":
		train_dataset_obj = PhysioNet('data/physionet', train=True, 
										quantization = args.quantization,
										download=True, n_samples = min(10000, args.n), 
										device = device)
		# Use custom collate_fn to combine samples with arbitrary time observations.
		# Returns the dataset along with mask and time steps
		test_dataset_obj = PhysioNet('data/physionet', train=False, 
										quantization = args.quantization,
										download=True, n_samples = min(10000, args.n), 
										device = device)

		# Combine and shuffle samples from physionet Train and physionet Test
		total_dataset = train_dataset_obj[:len(train_dataset_obj)]

		if not args.classif:
			# Concatenate samples from original Train and Test sets
			# Only 'training' physionet samples are have labels. Therefore, if we do classifiction task, we don't need physionet 'test' samples.
			total_dataset = total_dataset + test_dataset_obj[:len(test_dataset_obj)]

		# Shuffle and split
		train_data, test_data = model_selection.train_test_split(total_dataset, train_size= 0.8, 
			random_state = 42, shuffle = True)

		record_id, tt, vals, mask, labels = train_data[0]

		n_samples = len(total_dataset)
		input_dim = vals.size(-1)

		batch_size = min(min(len(train_dataset_obj), args.batch_size), args.n)
		data_min, data_max = get_data_min_max(total_dataset)

		train_dataloader = DataLoader(train_data, batch_size= batch_size, shuffle=False, 
			collate_fn= lambda batch: variable_time_collate_fn(batch, args, device, data_type = "train",
				data_min = data_min, data_max = data_max))
		test_dataloader = DataLoader(test_data, batch_size = n_samples, shuffle=False, 
			collate_fn= lambda batch: variable_time_collate_fn(batch, args, device, data_type = "test",
				data_min = data_min, data_max = data_max))

		attr_names = train_dataset_obj.params
		data_objects = {"dataset_obj": train_dataset_obj, 
					"train_dataloader": utils.inf_generator(train_dataloader), 
					"test_dataloader": utils.inf_generator(test_dataloader),
					"input_dim": input_dim,
					"n_train_batches": len(train_dataloader),
					"n_test_batches": len(test_dataloader),
					"attr": attr_names, #optional
					"classif_per_tp": False, #optional
					"n_labels": 1} #optional
		return data_objects

	##################################################################
	# Human activity dataset

	if dataset_name == "activity":
		n_samples =  min(10000, args.n)
		dataset_obj = PersonActivity('data/PersonActivity', 
							download=True, n_samples =  n_samples, device = device)
		print(dataset_obj)
		# Use custom collate_fn to combine samples with arbitrary time observations.
		# Returns the dataset along with mask and time steps

		# Shuffle and split
		train_data, test_data = model_selection.train_test_split(dataset_obj, train_size= 0.8, 
			random_state = 42, shuffle = True)

		train_data = [train_data[i] for i in np.random.choice(len(train_data), len(train_data))]
		test_data = [test_data[i] for i in np.random.choice(len(test_data), len(test_data))]

		record_id, tt, vals, mask, labels = train_data[0]
		input_dim = vals.size(-1)

		batch_size = min(min(len(dataset_obj), args.batch_size), args.n)
		train_dataloader = DataLoader(train_data, batch_size= batch_size, shuffle=False, 
			collate_fn= lambda batch: variable_time_collate_fn_activity(batch, args, device, data_type = "train"))
		test_dataloader = DataLoader(test_data, batch_size=n_samples, shuffle=False, 
			collate_fn= lambda batch: variable_time_collate_fn_activity(batch, args, device, data_type = "test"))

		data_objects = {"dataset_obj": dataset_obj, 
					"train_dataloader": utils.inf_generator(train_dataloader), 
					"test_dataloader": utils.inf_generator(test_dataloader),
					"input_dim": input_dim,
					"n_train_batches": len(train_dataloader),
					"n_test_batches": len(test_dataloader),
					"classif_per_tp": True, #optional
					"n_labels": labels.size(-1)}

		return data_objects

	########### 1d datasets ###########

	# Sampling args.timepoints time points in the interval [0, args.max_t]
	# Sample points for both training sequence and explapolation (test)
	distribution = uniform.Uniform(torch.Tensor([0.0]),torch.Tensor([max_t_extrap]))
	time_steps_extrap =  distribution.sample(torch.Size([n_total_tp-1]))[:,0]
	time_steps_extrap = torch.cat((torch.Tensor([0.0]), time_steps_extrap))
	time_steps_extrap = torch.sort(time_steps_extrap)[0]

	dataset_obj = None
	##################################################################
	# Sample a periodic function
	if dataset_name == "periodic":
		dataset_obj = Periodic_1d(
			init_freq = None, init_amplitude = 1.,
			final_amplitude = 1., final_freq = None, 
			z0 = 1.)

	##################################################################

	if dataset_obj is None:
		raise Exception("Unknown dataset: {}".format(dataset_name))

	dataset = dataset_obj.sample_traj(time_steps_extrap, n_samples = args.n, 
		noise_weight = args.noise_weight)

	# Process small datasets
	dataset = dataset.to(device)
	time_steps_extrap = time_steps_extrap.to(device)

	train_y, test_y = utils.split_train_test(dataset, train_fraq = 0.8)

	n_samples = len(dataset)
	input_dim = dataset.size(-1)

	batch_size = min(args.batch_size, args.n)
	train_dataloader = DataLoader(train_y, batch_size = batch_size, shuffle=False,
		collate_fn= lambda batch: basic_collate_fn(batch, time_steps_extrap, data_type = "train"))
	test_dataloader = DataLoader(test_y, batch_size = args.n, shuffle=False,
		collate_fn= lambda batch: basic_collate_fn(batch, time_steps_extrap, data_type = "test"))
	
	data_objects = {#"dataset_obj": dataset_obj, 
				"train_dataloader": utils.inf_generator(train_dataloader), 
				"test_dataloader": utils.inf_generator(test_dataloader),
				"input_dim": input_dim,
				"n_train_batches": len(train_dataloader),
				"n_test_batches": len(test_dataloader)}

	return data_objects


